In [1]:
import time
import os
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from typing import List
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests


email = 'kusainov.timur@gmail.com'
password = 'quhmen-Tywrim-hakwo5'
driver = webdriver.Safari()
wait = WebDriverWait(driver, 20)
driver.get('https://www.coursera.org/login')


def wait_load(by=By.ID, element='floating-ui-root'):
    element_present = EC.presence_of_element_located((by, element))
    time.sleep(10)
    wait.until(element_present)


def auth_coursera(email=email, paasword=password):
    
    username_field = driver.find_element(By.ID, 'email')  # Замените на актуальные селекторы
    password_field = driver.find_element(By.ID, 'password') 
    username_field.send_keys(email)
    password_field.send_keys(password)
    password_field.send_keys(Keys.ENTER)


def get_week_links():
    wait_load()
    nav_element = driver.find_element(By.XPATH, "//nav[@aria-label='Course']")
    nav_children = nav_element.find_elements(By.XPATH, ".//*")
    links = [element.get_attribute('href') for element in nav_children if element.text[:4] == 'Week' and element.get_attribute('href')]
    return [(f'week {i}', link) for i, link in enumerate(links, 1)]


def get_name_week():
    content = driver.find_element(By.XPATH, "//main[@class='course-content']")
    week_name = content.find_element(By.CLASS_NAME, 'cds-AccordionHeader-content').text.lower()
    week_name.replace(':', ' -')
    week_name = re.sub(r'[^a-zA-Z0-9\s-]', '', week_name)
    return week_name


def get_lessons():
    all_lesson = driver.find_element(By.CLASS_NAME, "rc-LessonCollectionBody")
    lesson_groups = all_lesson.find_elements(By.CLASS_NAME, 'rc-ItemGroupLesson')

    dict_lessons = {}
    
    for i, lesson_group in enumerate(lesson_groups, 1):
        lesson_group_name = lesson_group.find_element(By.CLASS_NAME, 'cds-AccordionHeader-labelGroup').text.lower()
        lesson_group_name = lesson_group_name.replace(':', ' -')
        lesson_group_name = lesson_group_name.replace('\xa0', ' ')
        lesson_group_name = re.sub(r'[^a-zA-Z0-9\s-]', '', lesson_group_name)
        lesson_group_name = f'{i}.' + lesson_group_name
        lessons_group = lesson_group.find_elements(By.XPATH, './/li')
        index_lesson = 1
        for lesson in lessons_group:
            lesson = lesson.find_element(By.TAG_NAME, 'a')
            if 'Video' in lesson.text:
                lesson_link = lesson.get_attribute('href')
                lesson_name = lesson.text.lower()
                lesson_name = lesson_name[:lesson_name.find('video')]
                lesson_name = lesson_name.replace(':', ' -')
                lesson_name = lesson_name.replace('\xa0', ' ')
                lesson_name = re.sub(r'[^a-zA-Z0-9\s-]', '', lesson_name)
                lesson_name = f'{index_lesson}.' + lesson_name
                index_lesson += 1
                dict_lessons.setdefault(lesson_group_name, []).append((lesson_name, lesson_link))     
    return dict_lessons


def window_help():
    close = driver.find_element(By.CLASS_NAME, "c-modal-x-out")
    close = close.find_element(By.TAG_NAME, 'a')
    driver.execute_script("arguments[0].scrollIntoView();", close)
    driver.execute_script("arguments[0].click();", close)


def choose_english():
    ul_element = driver.find_element(By.XPATH, '//ul[@id="subtitle-menu"]')
    buttons = ul_element.find_elements(By.TAG_NAME, 'button')
    for button in buttons:
        if 'English' in button.text:
            driver.execute_script("arguments[0].scrollIntoView();", button)
            driver.execute_script("arguments[0].click();", button)


def open_subtitles_menu():
    button = driver.find_element(By.ID, "subtitle-menu-button")
    driver.execute_script("arguments[0].scrollIntoView();", button)
    driver.execute_script("arguments[0].click();", button)


def download_video(url, path_video, path_subtitles):
    driver.get(url)
    wait_load()
    open_subtitles_menu()
    choose_english()
    button = driver.find_element(By.ID, "downloads-dropdown-btn")
    driver.execute_script("arguments[0].scrollIntoView();", button)
    driver.execute_script("arguments[0].click();", button)
    ul_element = driver.find_element(By.XPATH, '//ul[@aria-label="Downloads Menu"]')
    all_a= ul_element.find_elements(By.TAG_NAME, 'a')
    
    for a in all_a:
        if '720' in a.text:
            url = a.get_attribute('href')
            response = requests.get(url, stream=True)
            with open(path_video, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
        if 'WebVTT' in a.text:
            url = a.get_attribute('href')
            response = requests.get(url, stream=True)
            with open(path_subtitles, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)

def create_folder(path):
    try:
        os.mkdir(path)
    except FileExistsError:
        print(f'Folder "{path}" already exists')
    else:
        print(f'Created folder "{path}"')


def load_videos(url, name_week):
    driver.get(url)
    wait_load()
    create_folder(name_week)
        
    dict_lessons = get_lessons()
    print('Got all video names and links')
    for lesson_group_name in dict_lessons:
        path_lesson_group = os.path.join(name_week, lesson_group_name)
        create_folder(path_lesson_group)
        for (lesson_name, lesson_link) in dict_lessons[lesson_group_name]:
            path_video = os.path.join(path_lesson_group, lesson_name+'.mp4')
            path_subtitles = os.path.join(path_lesson_group, lesson_name+'.vtt')
            download_video(lesson_link, path_video, path_subtitles)
            print(f'Downloaded video "{lesson_name}"')
    print(f'All the video downloaded - {len(dict_lessons)}')

In [2]:
auth_coursera()
wait_load()

In [ ]:
url_course = 'https://www.coursera.org/learn/machine-learning-probability-and-statistics/home/week/1'
PATH = '/Users/timurkusainov/Desktop/Machine_Learning/translate/courses/math/3. Probability'
create_folder(PATH)
os.chdir(PATH)

driver.get(url_course)
weeks = get_week_links()

for name_week, url in weeks:
    load_videos(url, name_week)